In [27]:
import pandas as pd
import plotly.express as px
import numpy as np
import missingno as msn
import datetime
from pandas_profiling import ProfileReport
import sklearn
pd.options.mode.chained_assignment = None  # default='warn'


In [54]:
df = pd.read_parquet("escooter_history_2022.parquet")

In [75]:
df_33 = df.groupby(pd.Grouper(key="datetime",freq="D")).agg({"holiday":"mean","workingday":"mean"}).reset_index()

In [77]:
df_33.head()

,datetime,holiday,workingday
0,2020-01-04,0.0,0.0
1,2020-01-05,0.0,0.0
2,2020-01-06,0.0,1.0
3,2020-01-07,0.0,1.0
4,2020-01-08,0.0,1.0


In [78]:
df_round = df_33.loc[(~df_33.workingday.isin([1,0]) | ~df_33.holiday.isin([0,1]))]
df_round.head()

,datetime,holiday,workingday
28,2020-02-01,0.0,0.000950
56,2020-02-29,0.0,0.001797
58,2020-03-02,0.0,0.997782
65,2020-03-09,0.0,0.998101
119,2020-05-02,0.0,0.000483


In [79]:
df_round = df_33.loc[(~df_33.workingday.isin([1,0]) | ~df_33.holiday.isin([0,1]))].round(1).reset_index(drop=True)
df_round

,datetime,holiday,workingday
0,2020-02-01,0.0,0.0
1,2020-02-29,0.0,0.0
2,2020-03-02,0.0,1.0
3,2020-03-09,0.0,1.0
4,2020-05-02,0.0,0.0
5,2020-05-04,0.0,1.0
6,2020-05-09,0.0,0.0
7,2020-06-01,1.0,0.0
8,2020-06-02,0.0,1.0
9,2020-06-06,0.0,0.0


In [80]:
df.loc[df.datetime.dt.date == "2020-02-01", "workingday"] = 0

In [35]:
df_round.count()

datetime      41
holiday       41
workingday    41
dtype: int64

In [52]:
#df.loc[df.datetime.dt.date.isin(df_round.datetime.dt.date), "workingday"] = df_round.workingday

In [60]:
#df.workingday.apply(lambda x: df_round.workingday if (x in df_round.datetime.dt.date) else df.workingday)

In [ ]:
#df.loc[(np.where(df.datetime.dt.date.isin(df_round.datetime.dt.date))), df.workingday] = df_round.workingday

In [57]:
df.loc[df.datetime.dt.date.isin(df_round.datetime.dt.date)][["datetime", "holiday", "workingday"]]

,datetime,holiday,workingday
34320,2020-02-01 00:00:25,0.0,1.0
34321,2020-02-01 00:01:11,0.0,1.0
34322,2020-02-01 00:01:21,0.0,0.0
34323,2020-02-01 00:02:18,0.0,0.0
34324,2020-02-01 00:02:24,0.0,0.0
...,...,...,...
3755410,2022-01-03 23:58:10,0.0,1.0
3755411,2022-01-03 23:58:28,0.0,1.0
3755412,2022-01-03 23:58:46,0.0,1.0
3755413,2022-01-03 23:59:16,0.0,1.0
